<a href="https://colab.research.google.com/github/andoanminhkhanh/trituenhantao/blob/main/AI_C%E1%BA%A3i_thi%E1%BB%87n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Tăng cường dữ liệu
datagen = ImageDataGenerator(
    rescale=1./255,  # Chuẩn hóa ảnh bằng cách chia cho 255
    rotation_range=20,  # Xoay ảnh trong khoảng 20 độ
    width_shift_range=0.2,  # Dịch chuyển ảnh theo chiều ngang trong khoảng 20%
    height_shift_range=0.2,  # Dịch chuyển ảnh theo chiều dọc trong khoảng 20%
    shear_range=0.2,  # Áp dụng biến dạng cắt xén
    zoom_range=0.2,  # Phóng to/thu nhỏ ảnh trong khoảng 20%
    horizontal_flip=True,  # Lật ngang ảnh
    fill_mode='nearest',  # Cách lấp đầy các điểm trống sau khi biến đổi ảnh
    validation_split=0.2  # Tách 20% dữ liệu ra làm tập kiểm định
)

train_generator = datagen.flow_from_directory(
    "/content/drive/MyDrive/chest_xray/train",  # Đường dẫn tới dữ liệu huấn luyện
    target_size=(150, 150),  # Kích thước ảnh đầu vào
    batch_size=32,  # Kích thước lô
    class_mode='binary',  # Phân loại nhị phân
    subset='training'  # Sử dụng cho tập huấn luyện
)

validation_generator = datagen.flow_from_directory(
    "/content/drive/MyDrive/chest_xray/train",  # Đường dẫn tới dữ liệu huấn luyện
    target_size=(150, 150),  # Kích thước ảnh đầu vào
    batch_size=32,  # Kích thước lô
    class_mode='binary',  # Phân loại nhị phân
    subset='validation'  # Sử dụng cho tập kiểm định
)

# Kiến trúc mô hình
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Lớp Dropout để ngăn ngừa overfitting
    Dense(1, activation='sigmoid')
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callback dừng sớm
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Huấn luyện mô hình
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=100,
    callbacks=[early_stopping]
)

# Đánh giá mô hình trên tập kiểm tra
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    "/content/drive/MyDrive/chest_xray/test",  # Đường dẫn tới dữ liệu kiểm tra
    target_size=(150, 150),  # Kích thước ảnh đầu vào
    batch_size=32,  # Kích thước lô
    class_mode='binary'
)

test_loss, test_acc = model.evaluate(test_generator)
print(f'Độ chính xác trên tập kiểm tra: {test_acc}')


Found 4187 images belonging to 2 classes.
Found 1045 images belonging to 2 classes.
Epoch 1/100
 77/130 [================>.............] - ETA: 12:11 - loss: 0.5227 - accuracy: 0.7642

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Kiểm tra đường dẫn
train_dir = '/content/drive/MyDrive/chest_xray/train'
test_dir = '/content/drive/MyDrive/chest_xray/test'

if not os.path.exists(train_dir):
    print(f"Đường dẫn {train_dir} không tồn tại.")
if not os.path.exists(test_dir):
    print(f"Đường dẫn {test_dir} không tồn tại.")

# Tăng cường dữ liệu
datagen = ImageDataGenerator(
    rescale=1./255,  # Chuẩn hóa ảnh bằng cách chia cho 255
    rotation_range=20,  # Xoay ảnh trong khoảng 20 độ
    width_shift_range=0.2,  # Dịch chuyển ảnh theo chiều ngang trong khoảng 20%
    height_shift_range=0.2,  # Dịch chuyển ảnh theo chiều dọc trong khoảng 20%
    shear_range=0.2,  # Áp dụng biến dạng cắt xén
    zoom_range=0.2,  # Phóng to/thu nhỏ ảnh trong khoảng 20%
    horizontal_flip=True,  # Lật ngang ảnh
    fill_mode='nearest',  # Cách lấp đầy các điểm trống sau khi biến đổi ảnh
    validation_split=0.2  # Tách 20% dữ liệu ra làm tập kiểm định
)

train_generator = datagen.flow_from_directory(
    train_dir,  # Đường dẫn tới dữ liệu huấn luyện
    target_size=(128, 128),  # Giảm kích thước ảnh đầu vào
    batch_size=32,  # Kích thước lô
    class_mode='binary',  # Phân loại nhị phân
    subset='training'  # Sử dụng cho tập huấn luyện
)

validation_generator = datagen.flow_from_directory(
    train_dir,  # Đường dẫn tới dữ liệu huấn luyện
    target_size=(128, 128),  # Giảm kích thước ảnh đầu vào
    batch_size=32,  # Kích thước lô
    class_mode='binary',  # Phân loại nhị phân
    subset='validation'  # Sử dụng cho tập kiểm định
)

# Sử dụng mô hình tiền huấn luyện VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Không train lại các lớp của mô hình gốc
base_model.trainable = False

# Kiến trúc mô hình
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Lớp Dropout để ngăn ngừa overfitting
    Dense(1, activation='sigmoid')
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callback dừng sớm
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Huấn luyện mô hình
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20,  # Giảm số epochs
    callbacks=[early_stopping]
)

# Đánh giá mô hình trên tập kiểm tra
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,  # Đường dẫn tới dữ liệu kiểm tra
    target_size=(128, 128),  # Giảm kích thước ảnh đầu vào
    batch_size=32,  # Kích thước lô
    class_mode='binary'
)

test_loss, test_acc = model.evaluate(test_generator)
print(f'Độ chính xác trên tập kiểm tra: {test_acc}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 4187 images belonging to 2 classes.
Found 1045 images belonging to 2 classes.
58889256/58889256 [==============================] - 2s 0us/step
Epoch 1/20
130/130 [==============================] - 1477s 11s/step - loss: 0.3391 - accuracy: 0.8650 - val_loss: 0.2457 - val_accuracy: 0.8896
Epoch 2/20
 21/130 [===>..........................] - ETA: 13:30 - loss: 0.2366 - accuracy: 0.9092

KeyboardInterrupt: 